# Jupyter Notebook: 08_geopandas_basics.ipynb

# ---

# # GeoPandas Basics

Today we'll learn how to work with **vector spatial data** (points, lines, polygons) using **GeoPandas**! 🗺️🐼

---

## Table of Contents

1. What is GeoPandas?
2. GeoDataFrames
3. Reading Shapefiles and GeoJSON
4. Plotting Spatial Data
5. Basic Spatial Operations
6. Coordinate Reference Systems (CRS)
7. Mini-Exercises

---

# 1. What is GeoPandas?

**GeoPandas** extends **pandas** to work with **geometries** (points, lines, polygons).

Install it if needed:

```bash
pip install geopandas
```

Import it:

```python
import geopandas as gpd
```

---

# 2. GeoDataFrames

A **GeoDataFrame** is like a pandas DataFrame, but it stores a special `geometry` column.

Let's create one manually:

```python
from shapely.geometry import Point

# Sample data
cities = ["Montreal", "Paris", "Sydney"]
latitudes = [45.5017, 48.8566, -33.8688]
longitudes = [-73.5673, 2.3522, 151.2093]

# Create Points
geometry = [Point(xy) for xy in zip(longitudes, latitudes)]

# Create GeoDataFrame
gdf = gpd.GeoDataFrame({"City": cities, "Latitude": latitudes, "Longitude": longitudes}, geometry=geometry)

gdf
```

---

# 3. Reading Shapefiles and GeoJSON

You can load real spatial datasets easily:

```python
# Example: world countries
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

world.head()
```

Preview the dataset:

```python
world.plot(figsize=(12,8))
```

---

# 4. Plotting Spatial Data

GeoPandas makes plotting easy with Matplotlib behind the scenes:

```python
# Basic map
world.plot()
```

Customize colors:

```python
world.plot(color="lightblue", edgecolor="black", figsize=(12,8))
```

Color by attribute:

```python
world.plot(column='continent', categorical=True, legend=True, figsize=(12,8))
```

---

# 5. Basic Spatial Operations

GeoPandas provides powerful spatial methods!

## 5.1 Filtering by Attribute

```python
# Select countries in Africa
africa = world[world['continent'] == 'Africa']
africa.plot(color='coral', figsize=(10,6))
```

## 5.2 Buffer (creating a zone around geometries)

```python
# Buffer cities by 5 degrees (big circles for demo)
buffered_cities = gdf.copy()
buffered_cities["geometry"] = buffered_cities.buffer(5)

buffered_cities.plot(alpha=0.5)
```

## 5.3 Intersection

```python
# Find intersection between buffered cities and world
intersection = gpd.overlay(world, buffered_cities, how='intersection')
intersection.plot(color='purple', alpha=0.5)
```

## 5.4 Distance Calculations

```python
# Distance from Montreal to other cities
montreal = gdf[gdf['City'] == 'Montreal'].geometry.iloc[0]
gdf["Distance_to_Montreal"] = gdf.distance(montreal)

print(gdf[["City", "Distance_to_Montreal"]])
```

⚠️ **Warning:** Distance units depend on CRS! (More below)

---

# 6. Coordinate Reference Systems (CRS)

Check the CRS of a GeoDataFrame:

```python
print(world.crs)
```

Reproject to another CRS:

```python
# Reproject to Mercator (meters)
world_merc = world.to_crs(epsg=3857)

# Plot reprojected
world_merc.plot(figsize=(12,8))
```

---

# 7. Mini-Exercises

### 7.1 Load the world dataset and filter countries in South America

```python
# Your code here
south_america = world[world['continent'] == 'South America']
south_america.plot(color='green', figsize=(10,6))
```

### 7.2 Plot points for Buenos Aires (Argentina) and Rio de Janeiro (Brazil)

```python
# Your code here
cities_sa = ["Buenos Aires", "Rio de Janeiro"]
lat_sa = [-34.6037, -22.9068]
lon_sa = [-58.3816, -43.1729]

geometry_sa = [Point(xy) for xy in zip(lon_sa, lat_sa)]

gdf_sa = gpd.GeoDataFrame({"City": cities_sa}, geometry=geometry_sa)

ax = south_america.plot(color='lightgreen', figsize=(10,6))
gdf_sa.plot(ax=ax, color='red', markersize=50)
```

### 7.3 Buffer Buenos Aires by 2 degrees and show on map

```python
# Your code here
buffered_ba = gdf_sa[gdf_sa['City'] == 'Buenos Aires'].buffer(2)

ax = south_america.plot(color='lightgreen', figsize=(10,6))
gdf_sa.plot(ax=ax, color='red', markersize=50)
buffered_ba.plot(ax=ax, color='blue', alpha=0.5)
```

---

# Congratulations! 🎉

You've learned how to work with **GeoDataFrames**, **spatial operations**, **CRS**, and **mapping** using **GeoPandas**!

Next we'll explore raster data with **rasterio**! 🛰️📈

---

# Quick Recap
- GeoDataFrames store geometries + attributes.
- Read shapefiles/GeoJSON easily.
- Plot maps with simple commands.
- Do spatial analysis: filtering, buffering, intersections.
- Handle projections (CRS) properly.

See you in the next notebook!


In [3]:
# Import dependencies
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point, box
from pyproj import CRS

# BASIC AND COMPARISONS

In [19]:
# Creating a GeoDataFrame by hand
my_nonspatial_df = pd.DataFrame(data=[{"some_variable": 38}])
print(my_nonspatial_df)
print(type(my_nonspatial_df))

# Define geometry
my_geometry = Point(1.5, 2.5)
print(my_geometry)
print(type(my_geometry))

# Define CRS
my_crs = CRS.from_epsg(4326)
print(my_crs)
print(type(my_crs))

   some_variable
0             38
<class 'pandas.core.frame.DataFrame'>
POINT (1.5 2.5)
<class 'shapely.geometry.point.Point'>
EPSG:4326
<class 'pyproj.crs.crs.CRS'>


In [20]:
# Gotta give the geometry as a list or gpd not happy!
my_gdf = gpd.GeoDataFrame(data=my_nonspatial_df, geometry=[my_geometry], crs=my_crs)

#
print(my_gdf)
print(type(my_gdf))

   some_variable         geometry
0             38  POINT (1.5 2.5)
<class 'geopandas.geodataframe.GeoDataFrame'>


In [ ]:
# Use the .explore() method of GeoDataFrame
my_gdf.explore()

In [31]:
# See what type of object it is
my_explore = my_gdf.explore()

#
print(my_explore)
print(type(my_explore))

<class 'folium.folium.Map'>


# Loading spatial data

In [36]:
# Download reptile distribution areas
aire_reptiles = gpd.read_file(filename="https://diffusion.mffp.gouv.qc.ca/Diffusion/DonneeGratuite/Faune/Aires_repartition/Amphibien/SQLite/Aires_repartition_amphibiens.sqlite")

In [37]:
# See the first lines of the object
# No UTF8
aire_reptiles.head()

,desc_entit,producteur,nom_franca,nom_angla,nom_scient,date_maj,grand_groupe,famille,shape_length,shape_area,geometry
0,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Crapaud d'AmÃ©rique,American Toad,Anaxyrus americanus,2021,Amphibiens,Bufonidae,1.498345e+07,1.184087e+12,"MULTIPOLYGON (((-485026.718 132619.355, -48500..."
1,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Grenouille des bois,Wood Frog,Lithobates sylvaticus,2021,Amphibiens,Ranidae,1.332345e+07,1.198792e+12,"MULTIPOLYGON (((-485026.718 132619.355, -48500..."
2,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Grenouille des marais,Pickerel Frog,Lithobates palustris,2021,Amphibiens,Ranidae,5.387379e+06,1.573200e+11,"MULTIPOLYGON (((-485026.718 132619.355, -48500..."
3,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Grenouille du Nord,Mink Frog,Lithobates septentrionalis,2021,Amphibiens,Ranidae,1.106284e+07,9.748393e+11,"MULTIPOLYGON (((-485026.718 132619.355, -48500..."
4,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Grenouille lÃ©opard du Nord,Northern leopard Frog,Lithobates pipiens,2021,Amphibiens,Ranidae,9.757220e+06,6.338660e+11,"MULTIPOLYGON (((-485026.718 132619.355, -48500..."


In [46]:
# See distribution of the first species
#aire_reptiles.iloc[0:1].explore()

In [ ]:
# Create and append a new (it already exists but we want to show)
aire_reptiles["area_now"] = aire_reptiles.geometry.area

# Look at it!
aire_reptiles.head()

,desc_entit,producteur,nom_franca,nom_angla,nom_scient,date_maj,grand_groupe,famille,shape_length,shape_area,geometry,area_now
0,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Crapaud d'AmÃ©rique,American Toad,Anaxyrus americanus,2021,Amphibiens,Bufonidae,1.498345e+07,1.184087e+12,"MULTIPOLYGON (((-485026.718 132619.355, -48500...",1.184087e+12
1,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Grenouille des bois,Wood Frog,Lithobates sylvaticus,2021,Amphibiens,Ranidae,1.332345e+07,1.198792e+12,"MULTIPOLYGON (((-485026.718 132619.355, -48500...",1.198792e+12
2,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Grenouille des marais,Pickerel Frog,Lithobates palustris,2021,Amphibiens,Ranidae,5.387379e+06,1.573200e+11,"MULTIPOLYGON (((-485026.718 132619.355, -48500...",1.573200e+11
3,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Grenouille du Nord,Mink Frog,Lithobates septentrionalis,2021,Amphibiens,Ranidae,1.106284e+07,9.748393e+11,"MULTIPOLYGON (((-485026.718 132619.355, -48500...",9.748393e+11
4,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Grenouille lÃ©opard du Nord,Northern leopard Frog,Lithobates pipiens,2021,Amphibiens,Ranidae,9.757220e+06,6.338660e+11,"MULTIPOLYGON (((-485026.718 132619.355, -48500...",6.338660e+11


In [58]:
(aire_reptiles["area_now"] - aire_reptiles["shape_area"]).max()

np.float64(0.0093994140625)

In [67]:
aire_reptiles.groupby("famille")["area_now"].agg(["mean", "std", "count"])

,mean,std,count
famille,,,
Ambystomatidae,5.696182e+11,1.789843e+11,2
Bufonidae,1.184087e+12,NaN,1
Hylidae,1.867687e+11,3.299046e+11,4
Plethodontidae,2.289025e+11,3.293515e+11,6
Proteidae,2.150582e+10,NaN,1
Ranidae,6.292147e+11,4.009284e+11,6
Salamandridae,2.646686e+11,NaN,1


# Clipping

In [83]:
# Define a bounding box centered over northern Quebec
my_bbox = box(minx=-80.18, miny=51.24, maxx=-70.32, maxy=61.14)


# Create a GeoDataFrame
boxy = gpd.GeoDataFrame(data=[{"name": "boxy"}], geometry=[my_bbox], crs="EPSG:4326")
boxy

# Look at the bounding box
#boxy.explore()

,name,geometry
0,boxy,"POLYGON ((-70.32 51.24, -70.32 61.14, -80.18 6..."


In [120]:
# Clip the random polygons with the box
reptiles_clipped = gpd.clip(gdf=aire_reptiles, mask=boxy, sort=True)

C:\Users\elami\AppData\Local\Temp\ipykernel_25876\3823895502.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:32198
Right CRS: EPSG:4326

  reptiles_clipped = gpd.clip(gdf=aire_reptiles, mask=boxy, sort=True)


In [121]:
# AN ON PURPOSE ERROR!
# Reproject the left geometries to the appropriate CRS (EPSG:4326)
# Do it in the same line to avoid creating additional objects
reptiles_clipped = gpd.clip(gdf=aire_reptiles.to_crs(epsg=4326), mask=boxy, sort=True)

In [123]:
# Everything went well, look at the first few lines
reptiles_clipped.head()

,desc_entit,producteur,nom_franca,nom_angla,nom_scient,date_maj,grand_groupe,famille,shape_length,shape_area,geometry,area_now
0,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Crapaud d'AmÃ©rique,American Toad,Anaxyrus americanus,2021,Amphibiens,Bufonidae,1.498345e+07,1.184087e+12,"POLYGON ((-79.51779 51.25005, -79.51784 51.316...",1.184087e+12
1,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Grenouille des bois,Wood Frog,Lithobates sylvaticus,2021,Amphibiens,Ranidae,1.332345e+07,1.198792e+12,"POLYGON ((-79.51779 51.25005, -79.51784 51.316...",1.198792e+12
3,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Grenouille du Nord,Mink Frog,Lithobates septentrionalis,2021,Amphibiens,Ranidae,1.106284e+07,9.748393e+11,"POLYGON ((-79.51779 51.25005, -79.51784 51.316...",9.748393e+11
4,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Grenouille lÃ©opard du Nord,Northern leopard Frog,Lithobates pipiens,2021,Amphibiens,Ranidae,9.757220e+06,6.338660e+11,"POLYGON ((-78.80668 53.76057, -78.66113 53.766...",6.338660e+11
5,Aire de rÃ©partition,"MinistÃ¨re des ForÃªts, de la Faune et des Parcs",Grenouille verte,Green Frog,Lithobates clamitans,2021,Amphibiens,Ranidae,9.041373e+06,5.408896e+11,"POLYGON ((-74.73656 51.25987, -74.49898 51.337...",5.408896e+11


In [126]:
# Have a look at the data and review folium concepts
from folium import GeoJson
my_map = boxy.explore(color="red", alpha=0.75)
GeoJson(data=aire_reptiles[0:1].to_crs(epsg=4326).geometry, tooltip="I was not clipped!", color="purple", alpha=0.75).add_to(parent=my_map)
GeoJson(data=reptiles_clipped[0:1].geometry, tooltip="But I was!", color="blue", alpha=0.75).add_to(parent=my_map)
#my_map

# SPATIAL JOINS

In [2]:
# Get australian territories and boundaries
gdf_aus = gpd.read_file(filename="https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/STE_2021_AUST_SHP_GDA2020.zip")

# Look 
print(gdf_aus.head())

# Have a look at the shape of the GDF
# Note: I hate you last line...
print(gdf_aus.shape)

  STE_CODE21         STE_NAME21 CHG_FLAG21  CHG_LBL21 AUS_CODE21 AUS_NAME21  \
0          1    New South Wales          0  No change        AUS  Australia   
1          2           Victoria          0  No change        AUS  Australia   
2          3         Queensland          0  No change        AUS  Australia   
3          4    South Australia          0  No change        AUS  Australia   
4          5  Western Australia          0  No change        AUS  Australia   

     AREASQKM21                                       LOCI_URI21  \
0  8.007977e+05  http://linked.data.gov.au/dataset/asgsed3/STE/1   
1  2.274962e+05  http://linked.data.gov.au/dataset/asgsed3/STE/2   
2  1.730171e+06  http://linked.data.gov.au/dataset/asgsed3/STE/3   
3  9.842314e+05  http://linked.data.gov.au/dataset/asgsed3/STE/4   
4  2.526632e+06  http://linked.data.gov.au/dataset/asgsed3/STE/5   

                                            geometry  
0  MULTIPOLYGON (((159.0623 -31.50886, 159.06218 ...  
1  MUL

In [3]:
# Have a look at the CRS
print(gdf_aus.crs)
print(gdf_aus.crs.is_projected)
print(gdf_aus.crs.area_of_use)
print(type(gdf_aus.crs))

EPSG:7844
False
- name: Australia including Lord Howe Island, Macquarie Island, Ashmore and Cartier Islands, Christmas Island, Cocos (Keeling) Islands, Norfolk Island. All onshore and offshore.
- bounds: (93.41, -60.55, 173.34, -8.47)
<class 'pyproj.crs.crs.CRS'>


In [7]:
# Read in GBIF data
desert_pea = pd.read_csv(filepath_or_buffer="../data/desert_pea.csv")

# Look at the first 5 lines
desert_pea.head()

,Unnamed: 0,eventId,decimalLatitude,decimalLongitude
0,0,5087759453,-20.585195,116.787842
1,1,5104528592,-22.843242,122.580642
2,2,4510177813,-32.524942,136.163344
3,3,4512283168,-30.397483,136.874328
4,4,4516234716,-32.268875,135.993515


In [8]:
# Create a GDF from the coordinates
# Convert the latitude and longitude to Shapely Point geometries
pea_geom = [Point(lon, lat) for lon, lat in zip(desert_pea["decimalLongitude"], desert_pea["decimalLatitude"])]

# Look at the first 5 values of WKT Points
pea_geom[0:5]

[<POINT (116.788 -20.585)>,
 <POINT (122.581 -22.843)>,
 <POINT (136.163 -32.525)>,
 <POINT (136.874 -30.397)>,
 <POINT (135.994 -32.269)>]

In [9]:
# Create a GDF from the two
gdf_pea = gpd.GeoDataFrame(data=desert_pea, geometry=pea_geom)

# Observe the first 5 values
gdf_pea.head()

,Unnamed: 0,eventId,decimalLatitude,decimalLongitude,geometry
0,0,5087759453,-20.585195,116.787842,POINT (116.78784 -20.5852)
1,1,5104528592,-22.843242,122.580642,POINT (122.58064 -22.84324)
2,2,4510177813,-32.524942,136.163344,POINT (136.16334 -32.52494)
3,3,4512283168,-30.397483,136.874328,POINT (136.87433 -30.39748)
4,4,4516234716,-32.268875,135.993515,POINT (135.99352 -32.26888)


In [10]:
# BUT! Look at the CRS
print(gdf_pea.crs)

None


In [11]:
# We need to manually set the CRS.
# GBIF considers WGS84 (i.e. EPSG:4326 for their data)
gdf_pea.set_crs(crs="EPSG:4326", inplace=True)

# Now it will work
print(gdf_pea.crs)

EPSG:4326


In [17]:
###################################NOTICE
#
# Convert to GeoDataFrame in one go using from_xy
# But the previous method shows each step and is therefore more
# appropriate for a tutorial (yep!)
#
gdf_pea = gpd.GeoDataFrame(data=desert_pea["eventId"], geometry=gpd.points_from_xy(x=desert_pea.decimalLongitude, y=desert_pea.decimalLatitude, crs=4326))
print(gdf_pea)

        eventId                     geometry
0    5087759453   POINT (116.78784 -20.5852)
1    5104528592  POINT (122.58064 -22.84324)
2    4510177813  POINT (136.16334 -32.52494)
3    4512283168  POINT (136.87433 -30.39748)
4    4516234716  POINT (135.99352 -32.26888)
..          ...                          ...
295  3985803395  POINT (136.88933 -30.57152)
296  3986318782  POINT (136.92632 -30.42622)
297  3985793950  POINT (136.55266 -31.27423)
298  3985875946  POINT (136.86227 -33.25164)
299  3985925954  POINT (136.54971 -31.25694)

[300 rows x 2 columns]


In [18]:
# Now convert it to the same CRS as that of the AUSTRALIA DATA
gdf_pea.to_crs(crs=7844, inplace=True)

# Check that it has been correctly changed
print(gdf_pea.crs)

EPSG:7844


In [19]:
# Now you can perform spatial joins between `points_gdf` and `territories_gdf`
# For example, join the points with the Australian territories based on location
joined_gdf = gpd.sjoin(left_df=gdf_pea, right_df=gdf_aus, how="left", predicate="within")

# Display the resulting GeoDataFrame
print(joined_gdf)

        eventId                     geometry  index_right STE_CODE21  \
0    5087759453   POINT (116.78784 -20.5852)          4.0          5   
1    5104528592  POINT (122.58064 -22.84324)          4.0          5   
2    4510177813  POINT (136.16334 -32.52494)          3.0          4   
3    4512283168  POINT (136.87433 -30.39748)          3.0          4   
4    4516234716  POINT (135.99352 -32.26888)          3.0          4   
..          ...                          ...          ...        ...   
295  3985803395  POINT (136.88933 -30.57152)          3.0          4   
296  3986318782  POINT (136.92632 -30.42622)          3.0          4   
297  3985793950  POINT (136.55266 -31.27423)          3.0          4   
298  3985875946  POINT (136.86227 -33.25164)          3.0          4   
299  3985925954  POINT (136.54971 -31.25694)          3.0          4   

            STE_NAME21 CHG_FLAG21  CHG_LBL21 AUS_CODE21 AUS_NAME21  \
0    Western Australia          0  No change        AUS  Australi

In [27]:
# VERY IMPORTANT COLUMN
# INDEX_RIGHT
print(joined_gdf.index_right)

0      4.0
1      4.0
2      3.0
3      3.0
4      3.0
      ... 
295    3.0
296    3.0
297    3.0
298    3.0
299    3.0
Name: index_right, Length: 300, dtype: float64


In [29]:
# 
print(joined_gdf.shape)
#
print(joined_gdf.index_right.value_counts())
#
print(joined_gdf.index_right.value_counts().sum())

(300, 11)
index_right
4.0    106
3.0     94
0.0     74
6.0     24
Name: count, dtype: int64
298


In [38]:
mapy = gdf_aus.explore()

#for idx, point in joined_gdf.iterrows():
for _, point in joined_gdf.iterrows():
    folium.Marker(
        location=[point.geometry.y, point.geometry.x],
        popup=point.eventId,
        icon=folium.Icon(color="green", prefix="fa", icon="seedling")
    ).add_to(mapy)

# Display the map
#mapy

In [41]:
# See which points did not make the cut and why
sad_points = joined_gdf[joined_gdf["index_right"].isna()]

print(sad_points)

        eventId                     geometry  index_right STE_CODE21  \
147  4413908515  POINT (118.63845 -20.30648)          NaN        NaN   
284  3947437828   POINT (116.59086 -20.4657)          NaN        NaN   

    STE_NAME21 CHG_FLAG21 CHG_LBL21 AUS_CODE21 AUS_NAME21  AREASQKM21  \
147        NaN        NaN       NaN        NaN        NaN         NaN   
284        NaN        NaN       NaN        NaN        NaN         NaN   

    LOCI_URI21  
147        NaN  
284        NaN  


In [ ]:
mapo = gdf_aus.explore()

#for idx, point in sad_points.iterrows():
for _, point in sad_points.iterrows():
    folium.Marker(
        location=[point.geometry.y, point.geometry.x],
        popup=point.eventId,
        icon=folium.Icon(color="red", prefix="fa", icon="plant-wilt")
    ).add_to(mapo)

# Display the map
#mapo

In [43]:
# Group by the territory (or other relevant column in territories_gdf) and count occurrences
# Assuming the relevant column in territories is called 'territory_name' or similar
# If you have a different column name for the class/territory, replace 'territory_name' with that column name

territory_counts = joined_gdf.groupby("STE_NAME21").size()

# Step 5: Print the counts of observations per territory
print("\nUnsorted values:")
print(territory_counts)

# Note: You could also sort the counts in descending order
print("\nSorted values:")
print(territory_counts.sort_values(ascending=False))


Unsorted values:
STE_NAME21
New South Wales        74
Northern Territory     24
South Australia        94
Western Australia     106
dtype: int64

Sorted values:
STE_NAME21
Western Australia     106
South Australia        94
New South Wales        74
Northern Territory     24
dtype: int64


In [ ]:
##################### JUST FOR KICKS?
# Now you can perform spatial joins between `points_gdf` and `territories_gdf`
# For example, join the points with the Australian territories based on location
#joined_gdf = gpd.sjoin(left_df=gdf_pea, right_df=gdf_aus, how="right", predicate="within")
joined_gdf = gpd.sjoin(left_df=gdf_aus, right_df=gdf_pea, how="left", predicate="contains")

# Display the resulting GeoDataFrame
#print(joined_gdf)

# BE CAREFUL OF ONE THING
print(joined_gdf.STE_NAME21.value_counts())

#print(joined_gdf.tail())

# Don't forget to look at the index_right column!!!!
# It ain't 1 cuz they only got 1 observation!!!
joined_gdf[joined_gdf["index_right"].isna()]

STE_NAME21
Western Australia               106
South Australia                  94
New South Wales                  74
Northern Territory               24
Victoria                          1
Queensland                        1
Tasmania                          1
Australian Capital Territory      1
Other Territories                 1
Outside Australia                 1
Name: count, dtype: int64


,STE_CODE21,STE_NAME21,CHG_FLAG21,CHG_LBL21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry,index_right,Unnamed: 0,eventId,decimalLatitude,decimalLongitude
1,2,Victoria,0,No change,AUS,Australia,2.274962e+05,http://linked.data.gov.au/dataset/asgsed3/STE/2,"MULTIPOLYGON (((146.29286 -39.15778, 146.29341...",NaN,NaN,NaN,NaN,NaN
2,3,Queensland,0,No change,AUS,Australia,1.730171e+06,http://linked.data.gov.au/dataset/asgsed3/STE/3,"MULTIPOLYGON (((142.5314 -10.68301, 142.53072 ...",NaN,NaN,NaN,NaN,NaN
5,6,Tasmania,0,No change,AUS,Australia,6.801754e+04,http://linked.data.gov.au/dataset/asgsed3/STE/6,"MULTIPOLYGON (((144.60439 -41.01001, 144.60443...",NaN,NaN,NaN,NaN,NaN
7,8,Australian Capital Territory,0,No change,AUS,Australia,2.358133e+03,http://linked.data.gov.au/dataset/asgsed3/STE/8,"POLYGON ((149.06239 -35.1591, 149.09134 -35.14...",NaN,NaN,NaN,NaN,NaN
8,9,Other Territories,0,No change,AUS,Australia,2.557420e+02,http://linked.data.gov.au/dataset/asgsed3/STE/9,"MULTIPOLYGON (((167.94747 -29.12757, 167.94748...",NaN,NaN,NaN,NaN,NaN
9,Z,Outside Australia,1,New,ZZZ,Outside Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/STE/Z,None,NaN,NaN,NaN,NaN,NaN


In [31]:
import pandas as pd

pd.DataFrame({"eventId": ids, "decimalLatitude": latitudes, "decimalLongitude": longitudes}).to_csv(path_or_buf="desert_pea.csv")

# Case 2: Dugongs

In [4]:
marine_gdf = gpd.read_file(filename="https://hub.arcgis.com/api/v3/datasets/2b3eb1d42b8d4319900cf4777f0a83b9_0/downloads/data?format=shp&spatialRefId=4283&where=1%3D1")

print(marine_gdf.crs)
print(marine_gdf.crs.is_projected)
print(marine_gdf.crs.area_of_use)

EPSG:4283
False
- name: Australia including Lord Howe Island, Macquarie Island, Ashmore and Cartier Islands, Christmas Island, Cocos (Keeling) Islands, Norfolk Island. All onshore and offshore.
- bounds: (93.41, -60.55, 173.34, -8.47)


In [ ]:
# Have a look
marine_gdf.to_crs(crs=3577).explore()

In [8]:

# This time quick
df_dugong = pd.read_csv(filepath_or_buffer="../data/sea_cow.csv")
gdf_dugong = gpd.GeoDataFrame(data=df_dugong.eventId, geometry=gpd.points_from_xy(df_dugong.decimalLongitude, df_dugong.decimalLatitude, crs=4326))

# Get also Australian marine parks
gdf_marineparks = gpd.read_file(filename="https://hub.arcgis.com/api/v3/datasets/2b3eb1d42b8d4319900cf4777f0a83b9_0/downloads/data?format=shp&spatialRefId=4283&where=1%3D1")

# Safety check: verify both CRS (you set the first one though...)
print(gdf_dugong.crs)
print(gdf_dugong.crs.is_projected)
print(gdf_dugong.crs.area_of_use)
#
print("-" * 50)
#
print(gdf_marineparks.crs)
print(gdf_marineparks.crs.is_projected)
print(gdf_marineparks.crs.area_of_use)

EPSG:4326
False
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
--------------------------------------------------
EPSG:4283
False
- name: Australia including Lord Howe Island, Macquarie Island, Ashmore and Cartier Islands, Christmas Island, Cocos (Keeling) Islands, Norfolk Island. All onshore and offshore.
- bounds: (93.41, -60.55, 173.34, -8.47)


In [9]:
# Reproject both GeoDataFrames to the same projected CRS (EPSG:7855)
#gdf_dugong.to_crs(epsg=7855, inplace=True)
#gdf_marineparks.to_crs(epsg=7855, inplace=True)

gdf_dugong.to_crs(epsg=3577, inplace=True)
gdf_marineparks.to_crs(epsg=3577, inplace=True)

# Safety check: verify both CRS (you set the first one though...)
print(gdf_dugong.crs)
print(gdf_dugong.crs.is_projected)
print(gdf_dugong.crs.area_of_use)
#
print("-" * 50)
#
print(gdf_marineparks.crs)
print(gdf_marineparks.crs.is_projected)
print(gdf_marineparks.crs.area_of_use)

EPSG:3577
True
- name: Australia - Australian Capital Territory; New South Wales; Northern Territory; Queensland; South Australia; Tasmania; Western Australia; Victoria.
- bounds: (112.85, -43.7, 153.69, -9.86)
--------------------------------------------------
EPSG:3577
True
- name: Australia - Australian Capital Territory; New South Wales; Northern Territory; Queensland; South Australia; Tasmania; Western Australia; Victoria.
- bounds: (112.85, -43.7, 153.69, -9.86)


In [ ]:
# Create a buffer around the marine park. For example, 10 km buffer
#gdf_mpbuff = gpd.GeoDataFrame(data=gdf_marineparks, geometry=gdf_marineparks.buffer(distance=40000), crs=7855)
gdf_mpbuff = gpd.GeoDataFrame(data=gdf_marineparks, geometry=gdf_marineparks.buffer(distance=40000), crs=3577)

# Have a look at the data
gdf_mpbuff.explore()

In [12]:
# 3. Perform a spatial join to find dugong sightings within the buffer
# We use 'intersects' to check if a dugong sighting is within the buffer
join_dugong = gpd.sjoin(left_df=gdf_dugong, right_df=gdf_mpbuff, how="left", predicate="intersects")

# 4. Count the number of sightings within the buffer
# NONONONO
print(join_dugong.shape)

(309, 13)


In [13]:
# Be careful!
# Joins are done per cominations!!!!
join_dugong.eventId.value_counts()

eventId
4138745637    4
4405405111    3
4405391134    3
3391347177    2
4954727789    2
             ..
4015070427    1
4018147180    1
4519168632    1
4133921777    1
5075164945    1
Name: count, Length: 300, dtype: int64

In [ ]:
# Extract happy dugongs and sad dugongs
happy_dugongs = join_dugong.dropna(subset=["index_right"]).copy()
happy_dugongs.to_crs(4326, inplace=True)
#
sad_dugongs = join_dugong[join_dugong["index_right"].isna()].copy()
sad_dugongs.to_crs(4326, inplace=True)

# Map them both on the same map
mapa = gdf_mpbuff.explore()

for _, point in happy_dugongs.iterrows():
    folium.Marker(
        location=[point.geometry.y, point.geometry.x],
        popup=point.eventId,
        icon=folium.Icon(color="green", prefix="fa", icon="hippo")
    ).add_to(mapa)

for _, point in sad_dugongs.iterrows():
    folium.Marker(
        location=[point.geometry.y, point.geometry.x],
        popup=point.eventId,
        icon=folium.Icon(color="red", prefix="fa", icon="hippo")  # Optional: customize icon
    ).add_to(mapa)

# Display the map
mapa

In [75]:
join_dugong.columns

Index(['Unnamed: 0', 'eventId', 'decimalLatitude', 'decimalLongitude',
       'geometry', 'index_right', 'OBJECTID', 'NETNAME', 'RESNAME', 'ZONENAME',
       'ZONEIUCN', 'POLYGONID', 'NATLEGEND', 'AREA_KM2', 'SHAPEAREA',
       'SHAPELEN'],
      dtype='object')

In [91]:
join_dugong.dropna().groupby("ZONENAME").size().sort_values(ascending=False)

ZONENAME
Habitat Protection Zone         150
Multiple Use Zone                13
Recreational Use Zone             5
National Park Zone                3
Special Purpose Zone              1
Special Purpose Zone (Trawl)      1
dtype: int64

In [90]:
join_dugong.dropna().groupby("RESNAME").size().sort_values(ascending=False)

RESNAME
Limmen                   148
Roebuck                    8
Dampier                    6
Ningaloo                   6
Gascoyne                   3
Gulf of Carpentaria        1
Joseph Bonaparte Gulf      1
dtype: int64

In [94]:
join_dugong.dropna().groupby("POLYGONID").size().sort_values(ascending=False)

POLYGONID
nolimhpz01    148
nwroemuz01      8
nwgasmuz03      3
nwninruz01      3
nwdammuz03      2
nwdamnpz01      2
nwdamhpz02      2
nwninruz03      2
nogocspt02      1
nojbgspz01      1
nwninnpz02      1
dtype: int64